In [1]:
# Import Library
import numpy as np
import pandas as pd
import os
import sqlalchemy as db
import mysql.connector
from mysql.connector import errorcode
from sqlalchemy import create_engine
import pymysql
import xlrd
import decimal
from datetime import date, datetime

D = decimal.Decimal


In [2]:
data_path1 = 'C:\\Users\\fwidjojoputra\\Desktop\\data\\'
data_path2 = 'E:\\EASi\\5-Operations\\50-Management\\500-DataCentral\\3-Super User\\Development\\data_repository\\msp_data\\'

file_name1 = os.path.join(data_path1,'resources.csv') 
file_name2 = os.path.join(data_path1,'projects.csv') 
file_name3 = os.path.join(data_path1,'tasks.csv') 
file_name4 = os.path.join(data_path1,'assignments.csv') 
file_name5 = os.path.join(data_path1,'tasktime_phased.csv')
file_name6 = os.path.join(data_path2,'psoft_data.csv')

In [3]:
df1 = pd.read_csv(file_name1) # res
df2 = pd.read_csv(file_name2) # prj
df3 = pd.read_csv(file_name3) # task
df4 = pd.read_csv(file_name4) # ass
df5 = pd.read_csv(file_name5) # task_tp
df6 = pd.read_csv(file_name6) # psoft_data

In [4]:
#-------------------- Data prep RESOURCES --------------------#
res_df = df1.loc[:,['DataCentralID',
                    'ResourceId']]

prj_df = df2.loc[:,['ProjectId',
                    'SOWNb',
                    'ProjectActualFinishDate',
                    'year']]

tsk_df = df3.loc[:,['TaskId',
                    'ParentTaskId',
                    'ProjectId',
                    'TaskName']]

ass_df = df4.loc[:,['ProjectId',
                    'TaskId',
                    'AssignmentId',
                    'ResourceId']]

tsktp_df = df5.loc[:,['TaskId',
                      'ProjectId',
                      'TaskWork',
                      'TaskCost',
                      'TaskName']] 

psoft_df = df6.loc[:,['Customer',
                      'Category',
                      'Order No',
                      'Job Req #',
                      'ID',
                      'Name',
                      'Earn Code',
                      'Blended Rate',
                      'Job Code']]

res_df.rename({'DataCentralID':'easi_id',
               'ResourceId':'res_id'},axis=1, inplace=True)

prj_df.rename({'SOWNb':'sow_no',
               'ProjectId':'prj_id',
               'ProjectActualFinishDate':'end_date'},axis=1, inplace=True)

tsk_df.rename({'TaskId':'task_id',
               'ParentTaskId':'parent_id',
               'ProjectId':'prj_id',
               'TaskName':'task_name'},axis=1, inplace=True)

ass_df.rename({'ProjectId':'prj_id',
                'TaskId':'task_id',
                'AssignmentId':'ass_id',
                'ResourceId':'res_id'},axis=1, inplace=True)

tsktp_df.rename({'TaskId':'task_id',
                 'ProjectId':'prj_id',
                 'TaskWork':'task_work',
                 'TaskCost':'task_cost',
                 'TaskName':'task_name'},axis=1, inplace=True)

psoft_df.rename({'Customer':'client',
                 'Category':'cat',
                 'Order No':'ord_no',
                 'Job Req #':'job_req',
                 'ID':'easi_id',
                 'Name':'res_name',
                 'Earn Code':'earn_code',
                 'Blended Rate':'br',
                 'Job Code':'job_code'},axis=1, inplace=True)


In [5]:
prj_tsk_df = tsk_df.merge(prj_df, on='prj_id', how='left')
prj_task_ass_df = prj_tsk_df.merge(ass_df, on='task_id', how='inner')
prj_task_ass_res_df = prj_task_ass_df.merge(res_df, on='res_id', how='left')

# group dataset by task_id
task_timephased_df = tsktp_df.groupby('task_id')['task_work','task_cost'].sum()
tsktp_df1 = task_timephased_df.reset_index()
tsktp_df1['br'] = tsktp_df1['task_cost']/tsktp_df1['task_work']

all_df = prj_task_ass_res_df.merge(tsktp_df1, on='task_id', how='left')

In [6]:

temp1_df = all_df.loc[:,['sow_no',
                          'task_name',
                          'easi_id',
                          'task_cost',
                          'task_work',
                          'br']]
temp1_df = temp1_df.fillna(value='0')


In [9]:
# modify timephased dataset
temp1_df.head()
all_df.head()

,task_id,parent_id,prj_id_x,task_name,sow_no,end_date,year,prj_id_y,ass_id,res_id,easi_id,task_work,task_cost,br
0,dc166ecc-2f0d-e911-8197-d4258bdae0ca,db166ecc-2f0d-e911-8197-d4258bdae0ca,d7166ecc-2f0d-e911-8197-d4258bdae0ca,Engine Test Hardware,E1B100372792-A,NaN,2019.0,d7166ecc-2f0d-e911-8197-d4258bdae0ca,e0166ecc-2f0d-e911-8197-d4258bdae0ca,b067ce64-9588-4b18-bf54-3491f76419c9,NaN,0.0,0.0,NaN
1,dd166ecc-2f0d-e911-8197-d4258bdae0ca,db166ecc-2f0d-e911-8197-d4258bdae0ca,d7166ecc-2f0d-e911-8197-d4258bdae0ca,Engine Test Support,E1B100372792-A,NaN,2019.0,d7166ecc-2f0d-e911-8197-d4258bdae0ca,e1166ecc-2f0d-e911-8197-d4258bdae0ca,b067ce64-9588-4b18-bf54-3491f76419c9,NaN,0.0,0.0,NaN
2,51ecd179-2c73-e911-9b6e-d4258bdae0ca,db166ecc-2f0d-e911-8197-d4258bdae0ca,d7166ecc-2f0d-e911-8197-d4258bdae0ca,V&V support,E1B100372792-A,NaN,2019.0,d7166ecc-2f0d-e911-8197-d4258bdae0ca,17ab3fe5-2c73-e911-9b6e-d4258bdae0ca,c34cf8ec-b0de-e811-9640-d4258bdae0ca,05765755,670.5,59406.3,88.6
3,186647ca-330d-e911-8197-d4258bdae0ca,176647ca-330d-e911-8197-d4258bdae0ca,136647ca-330d-e911-8197-d4258bdae0ca,"Remastered FRE100594 drawing (new part number,...",E1B100392830-1,NaN,2019.0,136647ca-330d-e911-8197-d4258bdae0ca,236647ca-330d-e911-8197-d4258bdae0ca,b067ce64-9588-4b18-bf54-3491f76419c9,NaN,0.0,0.0,NaN
4,196647ca-330d-e911-8197-d4258bdae0ca,176647ca-330d-e911-8197-d4258bdae0ca,136647ca-330d-e911-8197-d4258bdae0ca,Final Issue FRE100594 drawing (new part number...,E1B100392830-1,NaN,2019.0,136647ca-330d-e911-8197-d4258bdae0ca,246647ca-330d-e911-8197-d4258bdae0ca,b067ce64-9588-4b18-bf54-3491f76419c9,NaN,0.0,0.0,NaN
